In [39]:
img_height = 128
img_width = 128
img_channels = 3
img_size = (img_height, img_width)
train_dir = r"/home/mwaniki-new/Documents/deep_learning/transfer_learning/src/train"
test_dir = r"/home/mwaniki-new/Documents/deep_learning/transfer_learning/src/test"
batch_size = 64

In [40]:
import comet_ml

comet_ml.init_project("transfer_learning_training")


COMET INFO: Comet API key is valid


In [41]:
from comet_ml import Experiment
#initializing comet experiment
experiment = Experiment()

parameters = {
    "batch_size":batch_size,
    "epochs": 5,
    "optimizer": "adam",
    "loss": "binary_crossentropy",
}

experiment.log_parameters(parameters)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/mwanikinjagi/general/8a3039fcf2f44d0e8fd1807d21cbdce0
COMET INFO:   Parameters:
COMET INFO:     batch_size : 64
COMET INFO:     epochs     : 5
COMET INFO:     loss       : binary_crossentropy
COMET INFO:     optimizer  : adam
COMET INFO:   Uploads:


COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: ---------------------------
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/home/mwaniki-new/Documents/deep_learning/transfer_learning' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/mwanikinjagi/general/87f45e8c992c44b896f8a64910ccf3da



In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator


#Importing dataframe
df = pd.read_csv("training_info.csv")

#performing training and validation splits
train_df, validation_df = train_test_split(df,test_size=0.2, random_state=42)

#preparing data generators
train_datagen = ImageDataGenerator(rotation_range=15,
                                   rescale=1./255,
                                   shear_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1)


train_gen = train_datagen.flow_from_dataframe(train_df, 
                                              train_dir, x_col='filename', y_col='category',
                                              target_size=img_size,
                                              class_mode='binary',
                                              batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)


validation_gen = validation_datagen.flow_from_dataframe(train_df, 
                                              train_dir, x_col='filename', y_col='category',
                                              target_size=img_size,
                                              class_mode='binary',
                                              batch_size=batch_size)

test_datagen = ImageDataGenerator(rotation_range=15,
                                  rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)
#we will make a test datagen later
#number of train and validation samples
total_train = train_df.shape[0]
total_validation = validation_df.shape[0]

Found 20000 validated image filenames belonging to 2 classes.
Found 20000 validated image filenames belonging to 2 classes.


In [43]:
total_train

20000

In [44]:
total_validation

5000

In [45]:
#model creation and training
from keras.applications.resnet import ResNet50
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.callbacks import EarlyStopping

#initialize sequential model
model = Sequential()

#base model
base = ResNet50(weights= 'imagenet',
                 include_top=False,
                 input_shape=(img_height, img_width, img_channels),
                 classes=2
                 )

for layer in base.layers:
    layer.trainable = False

#Adding layers to the model
model.add(base)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='softmax'))

#EarlyStopping callback
callback = EarlyStopping(monitor='loss',
                         patience=0)

In [46]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_4 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 512)               16777728  
                                                                 
 dense_9 (Dense)             (None, 1)                 513       
                                                                 
Total params: 40,365,953
Trainable params: 16,778,241
Non-trainable params: 23,587,712
_________________________________________________________________


In [47]:
model.compile(loss=parameters["loss"],
              optimizer=parameters["optimizer"],
              metrics=["accuracy"])

In [48]:
model.fit_generator(train_gen,
                    validation_data = validation_gen,
                    epochs=parameters["epochs"],
                    validation_steps=total_validation,
                    steps_per_epoch=total_train,
                    callbacks=[callback])

model.save("model_5_epoch.h5")

/home/mwaniki-new/.local/lib/python3.10/site-packages/comet_ml/monkey_patching.py:312: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  return_value = original(*args, **kwargs)


Epoch 1/5


2023-08-21 17:58:18.267449: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


    7/20000 [..............................] - ETA: 15:37:44 - loss: 1.9028 - accuracy: 0.5156

   14/20000 [..............................] - ETA: 16:23:55 - loss: 1.3820 - accuracy: 0.5067

In [ ]:
experiment.end()